<a href="https://colab.research.google.com/github/AliBank/VGG19/blob/main/Vgg19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q keras
!pip install h5py pyyaml

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.optimizers import RMSprop
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import load_img
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.applications.vgg19 import VGG19
import matplotlib.pyplot as plt
import cv2
import numpy as np
from glob import glob
from keras.utils import to_categorical
from __future__ import absolute_import, division, print_function
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D

In [ ]:
train_path="/content/drive/MyDrive/BaklagilDataSet/DB3_Histeq/fold1/train/images/"
test_path="/content/drive/MyDrive/BaklagilDataSet/DB3_Histeq/fold1/test/images/"

In [ ]:
numberOfClass=len(glob(train_path+"/*"))

In [ ]:
print(numberOfClass)

30


In [ ]:
vgg= VGG19()

574710816/574710816 [==============================] - 20s 0us/step


In [ ]:
print(vgg.summary())

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
print(type(vgg))

<class 'keras.engine.functional.Functional'>


In [ ]:
vgg_layer_list= vgg.layers
print(vgg_layer_list)

[<keras.engine.input_layer.InputLayer object at 0x7f48dbbfcbe0>, <keras.layers.convolutional.conv2d.Conv2D object at 0x7f48d918e130>, <keras.layers.convolutional.conv2d.Conv2D object at 0x7f4866fd61f0>, <keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7f48d918e160>, <keras.layers.convolutional.conv2d.Conv2D object at 0x7f4866fc9a60>, <keras.layers.convolutional.conv2d.Conv2D object at 0x7f48632aa4c0>, <keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7f4866fd6af0>, <keras.layers.convolutional.conv2d.Conv2D object at 0x7f48632aa760>, <keras.layers.convolutional.conv2d.Conv2D object at 0x7f486324f220>, <keras.layers.convolutional.conv2d.Conv2D object at 0x7f4863256f10>, <keras.layers.convolutional.conv2d.Conv2D object at 0x7f486325c220>, <keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7f48632c6b80>, <keras.layers.convolutional.conv2d.Conv2D object at 0x7f4863268190>, <keras.layers.convolutional.conv2d.Conv2D object at 0x7f4863268610>, <keras.layers.con

In [ ]:
model= Sequential()
for i in range(len(vgg_layer_list)-1):  
    model.add(vgg_layer_list[i])

print(model.summary())    


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [ ]:
for layers in model.layers:
    layers.trainable = False

In [ ]:
model.add(Dense(numberOfClass, activation="softmax"))

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [ ]:
from keras.optimizers.optimizer_v2.rmsprop import RMSProp
optim_1 = RMSProp(lr=0.0001)
model.compile(loss="categorical_crossentropy", optimizer=optim_1, metrics=["accuracy"])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
#train
train_data= ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224))
test_data= ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224))

Found 3601 images belonging to 30 classes.
Found 900 images belonging to 30 classes.


In [ ]:
batch_size=50

In [ ]:
from keras.callbacks import ModelCheckpoint,CSVLogger
filepath="/content/drive/MyDrive/saved_models/weights-improvementVgg19-{epoch:05d}.ckpt"
checkpoint= ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False,save_freq=5*72, mode='max')
#log_csv =CSVLogger('my_logs.csv', separator=',', append=False)
callbacks_list =[checkpoint]

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=100,
                         validation_data= test_data,
                         validation_steps = 900//batch_size )

<ipython-input-18-69e4907365a4>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist=model.fit_generator(train_data,


Epoch 1/100
72/72 [==============================] - 2243s 31s/step - loss: 3.2917 - accuracy: 0.1884 - val_loss: 2.1570 - val_accuracy: 0.3490
Epoch 2/100
72/72 [==============================] - 1935s 27s/step - loss: 1.4968 - accuracy: 0.5425 - val_loss: 1.3859 - val_accuracy: 0.5573
Epoch 3/100
72/72 [==============================] - 1903s 26s/step - loss: 1.0237 - accuracy: 0.6680 - val_loss: 1.1089 - val_accuracy: 0.6233
Epoch 4/100
72/72 [==============================] - 1894s 26s/step - loss: 0.7402 - accuracy: 0.7604 - val_loss: 0.8821 - val_accuracy: 0.7014
Epoch 5/100
72/72 [==============================] - 1892s 26s/step - loss: 0.5821 - accuracy: 0.8216 - val_loss: 0.7028 - val_accuracy: 0.7656
Epoch 6/100
72/72 [==============================] - 1890s 26s/step - loss: 0.4829 - accuracy: 0.8529 - val_loss: 0.6651 - val_accuracy: 0.7830
Epoch 7/100
72/72 [==============================] - 1899s 26s/step - loss: 0.4016 - accuracy: 0.8737 - val_loss: 0.6592 - val_accuracy:

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=15,
                         initial_epoch=0,
                         validation_data= test_data,
                         validation_steps = 900//batch_size,
                         callbacks=callbacks_list)

In [ ]:
model.load_weights("/content/drive/MyDrive/saved_models/weights-improvementVgg19-00015.ckpt")

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=25,
                         initial_epoch=15,
                         validation_data= test_data,
                         validation_steps = 900//batch_size,
                         callbacks=callbacks_list)

In [ ]:
model.load_weights("/content/drive/MyDrive/saved_models/weights-improvementVgg19-00025.ckpt")

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=40,
                         initial_epoch=25,
                         validation_data= test_data,
                         validation_steps = 900//batch_size,
                         callbacks=callbacks_list)

In [ ]:
model.load_weights("/content/drive/MyDrive/saved_models/weights-improvementVgg19-00040.ckpt")

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=50,
                         initial_epoch=40,
                         validation_data= test_data,
                         validation_steps = 900//batch_size,
                         callbacks=callbacks_list)

In [ ]:
model.load_weights("/content/drive/MyDrive/saved_models/weights-improvementVgg19-00050.ckpt")

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=75,
                         initial_epoch=50,
                         validation_data= test_data,
                         validation_steps = 900//batch_size,
                         callbacks=callbacks_list)

In [ ]:
model.load_weights("/content/drive/MyDrive/saved_models/weights-improvementVgg19-00075.ckpt")

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=100,
                         initial_epoch=75,
                         validation_data= test_data,
                         validation_steps = 900//batch_size,
                         callbacks=callbacks_list)

In [ ]:
model.load_weights("/content/drive/MyDrive/saved_models/weights-improvementVgg19-00100.ckpt")

In [ ]:
model.save_weights("BaklagilFold1RMSProplr0.0001Vgg19")